In [84]:
import openai
import os
import json
import threading
import pyaudio
import wave
import time
import geocoder
import numpy as np
import cv2
import tkinter as tk

from flask import Flask, render_template, request, jsonify, send_file
from dotenv import load_dotenv
from openai import OpenAI
from pathlib import Path
from datetime import datetime
from tkinter import messagebox

In [2]:
load_dotenv()
app = Flask(__name__)
openai.api_key = os.getenv('OPENAI_API_KEY')  # Replace with your actual OpenAI API key
client = OpenAI()

def get_location():
    g = geocoder.ip('me')
    city = g.city
    country = g.country
    zipcode = g.postal
    location_details = "You are in " + city + ", " + country + ". Your zipcode is " + zipcode + "."
    return location_details

In [66]:
def get_datetime():
    datetimeobject = datetime.now()
    date = datetimeobject.strftime("%d-%m-%Y")
    time = datetimeobject.strftime("%H:%M:%S")
    date_time = "Today's date is " + date + " and the time is " + time + "."
    return date_time

In [83]:
def capture_image():
    # Initialize the camera
    cap = cv2.VideoCapture(0)  # '0' is typically the default value for the laptop's built-in webcam

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    ret, frame = cap.read()
    if ret:
        # Save the captured image to a file
        cv2.imwrite('webcam_image.jpg', frame)
        print("Image captured and saved successfully.")
    else:
        print("Failed to capture image")

    # Release the camera
    cap.release()

In [85]:
def on_button_click():
    # Ask for permission to take a picture
    permission = messagebox.askokcancel("Permission", "Do you want to take a picture?")
    if permission:
        capture_image()

In [96]:
def use_camera():
    root = tk.Tk()
    root.title("Permission Request")

    button = tk.Button(root, text="Request Permission", command=on_button_click)
    button.pack(pady=20)

    root.mainloop()
    return("photo saved")

In [3]:
file1 = client.files.create(
    file=open("C:/Users/jatin/Downloads/SWE_Exhibitor_Information.pdf", "rb"),
    purpose='assistants'
)

In [88]:
assistant = client.beta.assistants.create(
        name="Exhibitor Assistant 1",
        instructions="You are an greeter at the SWE 2023 conference. Respond in a friendly manner.",
        model="gpt-4-1106-preview",
        tools=[
            {"type": "code_interpreter"},
            {"type":"retrieval"},
            {"type": "function",
             "function": {
                 "name": "get_location",
                 "description": "Get the location of the user.",
                 "parameters": {
                        "type": "object",
                        "properties": {
                                "user": {
                                "type": "string",
                                "description": "Name of the user"
                                },
                        },
                        "required": []
                }
            }
            },
            {"type": "function",
             "function": {
                 "name": "get_datetime",
                 "description": "Get the date and time of the user.",
                 "parameters": {
                        "type": "object",
                        "properties": {
                                "user": {
                                "type": "string",
                                "description": "Name of the user"
                                },
                        },
                        "required": []
                }
            }
            },
            {"type": "function",
             "function": {
                 "name": "use_camera",
                 "description": "Use the webcam camera to capture an image of the user or any object in the frame of the webcam.",
                 "parameters": {
                        "type": "object",
                        "properties": {
                                "user": {
                                "type": "string",
                                "description": "Name of the user"
                                },
                        },
                        "required": []
                }
            }
            },
        ],
        file_ids=[file1.id]
)

In [89]:
thread = client.beta.threads.create()

In [90]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="what location, date and time am I in? and take a picture of me",
)

In [91]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id
)

In [103]:
runStatus = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
print(runStatus.status)

completed


In [97]:
print(runStatus.required_action.submit_tool_outputs.tool_calls[2].function.name)

use_camera


In [98]:
print(len(runStatus.required_action.submit_tool_outputs.tool_calls))

3


In [77]:
msg = []
if runStatus.required_action.submit_tool_outputs.tool_calls[0].function.name == "get_location":
    msg.append(get_location())
    print(msg)
if runStatus.required_action.submit_tool_outputs.tool_calls[1].function.name == "get_datetime":
    msg.append(get_datetime())
    print(msg)

['You are in Livonia, US. Your zipcode is 48150.']
['You are in Livonia, US. Your zipcode is 48150.', "Today's date is 25-11-2023 and the time is 17:30:05."]


In [99]:
msg=[]
tool_calls = runStatus.required_action.submit_tool_outputs.tool_calls
for i in range(len(tool_calls)):
    if tool_calls[i].function.name == "get_location":
        msg.append({
            "tool_call_id": tool_calls[i].id,
            "output": get_location()
        })
    if tool_calls[i].function.name == "get_datetime":
        msg.append({
            "tool_call_id": tool_calls[i].id,
            "output": get_datetime()
        })
    if tool_calls[i].function.name == "use_camera":
        msg.append({
            "tool_call_id": tool_calls[i].id,
            "output": use_camera()
        })

print(msg)

Image captured and saved successfully.
[{'tool_call_id': 'call_kIxjWhBMsMy9BdLz2m9486s7', 'output': 'You are in Livonia, US. Your zipcode is 48150.'}, {'tool_call_id': 'call_4eYoB7Qz9mAXn5aCgnOQ7qHy', 'output': "Today's date is 25-11-2023 and the time is 17:43:15."}, {'tool_call_id': 'call_8Ebw3Qz6TjjYpVt8VLZurC20', 'output': 'photo saved'}]


In [100]:
run = client.beta.threads.runs.submit_tool_outputs(
  thread_id=thread.id,
  run_id=run.id,
  tool_outputs=msg
)

In [102]:
responses = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(responses.data[0].content[0].text.value)

You are currently in Livonia, US, with the zipcode 48150. The date today is November 25, 2023, and the time is 17:43:15. And here's the picture I've taken of you:

![Your Picture](sandbox:/mnt/data/photo.jpg)

Please let me know if there is anything else I can assist you with!


In [22]:
runStatus = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
)
print(runStatus)
print(runStatus.status)

Run(id='run_5z8wxOex3OlbCknco6s42G47', assistant_id='asst_EEBLH3jtrvnxEOsyqA257CUA', cancelled_at=None, completed_at=1700617250, created_at=1700617228, expires_at=None, failed_at=None, file_ids=['file-xZG5vQ6pOV6kAbdgKkZgnDOu'], instructions='You are an greeter at the SWE 2023 conference.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=1700617228, status='completed', thread_id='thread_uCJRH85vBY72HpPDwnPVjjMD', tools=[ToolAssistantToolsCode(type='code_interpreter'), ToolAssistantToolsRetrieval(type='retrieval')])
completed


In [34]:
responses = client.beta.threads.messages.list(
    thread_id=thread.id
)
print(responses)
print(responses.data[0].content)
print(responses.data[0].content[0].text.value)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_WqI8hfd0c8ctz39kLvLa4kuu', assistant_id='asst_EEBLH3jtrvnxEOsyqA257CUA', content=[MessageContentText(text=Text(annotations=[], value='The SWE 2023 conference, also known as WE23, is themed "Live Without Limits" and is scheduled to take place from October 26-28, 2023 in Los Angeles, California. Some highlights of the conference include:\n\n- Interactive workshops focused on leadership and engineering.\n- Opportunities for networking and gaining career insights.\n- Engagement with industry leaders on innovation.\n- Hands-on demonstrations and showcases of cutting-edge technology.\n- Panel discussions on emerging trends and best practices.\n- Sessions tailored for early-career professionals and aspiring leaders.\n- Emphasis on nurturing growth, networking, and career development.\n\nThis event is a part of the Society of Women Engineers\' mission to empower women in engineering and leadership, expand the image of the engineering pr

In [3]:
import os

current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

Current Working Directory: c:\Users\jatin\Documents\AI\base_1
